In [1]:
import numpy as np
import scipy.stats as st
import copy
import math
import matplotlib
import time
from matplotlib import pyplot as plt

class Oracle:
    def __init__(self, p, pi):
        assert(p.shape[1] == pi.shape[0])
        self.__pi = copy.copy(pi)
        self.__p = copy.copy(p)
        self.__calc = 0
        self.__grad = 0
        self.__gess = 0
    
    def Calculate(self, x):
        self.__calc += 1
        return np.sum(self.__pi * np.log(self.__p.T @ x))
    
    def CalculateGradient(self, x):
        self.__grad += 1
        return self.__p @ (self.__pi * (self.__p.T @ x)**(-1))
    
    def CalculateGessian(self, x):
        self.__gess += 1
        return -self.__p @ np.diagflat(self.__pi * (self.__p.T @ x)**(-2)) @ self.__p.T
    
    @property
    def Calculated(self):
        return self.__calc;
    
    @property
    def CalculatedGradients(self):
        return self.__grad;
    
    @property
    def CalculatedGessians(self):
        return self.__gess;
    
def Normalization(A):
    assert(A.shape[0] == A.shape[1])
    return A - np.eye(A.shape[0])
    
class QuasiNewtonIterator:
    def __init__(self, begin, oracle, projector, step_choser, priority, checker, normalizator):
        self.__point = copy.copy(begin)
        self.__oracle = copy.copy(oracle)
        self.__projector = copy.copy(projector)
        self.__step_choser = copy.copy(step_choser)
        self.__priority = copy.copy(priority)
        self.__checker = copy.copy(checker)
        self.__normalizator = copy.copy(normalizator)
        self.__step = 1.0
        self.__value = oracle.Calculate(begin)
        self.__grad = oracle.CalculateGradient(begin)
        self.__C = -np.eye(begin.size)
    def MakeStep(self):
        direction = self.__projector(self.__C @ self.__grad)
        old_value = self.__value
        dpoint = self.__point
        new_point = self.__point - self.__step * direction
        while (not self.__checker(new_point)):
            self.__step = self.__step_choser(self.__step, False)
            new_point = self.__point - self.__step * direction
        new_value = self.__oracle.Calculate(new_point)
        self.__step = self.__step_choser(self.__step, True)
        dpoint = new_point - self.__point
        self.__point = new_point
        self.__value = new_value
        new_grad = self.__oracle.CalculateGradient(self.__point)
        dgrad = new_grad - self.__grad
        self.__grad = new_grad
        dpoint = dpoint.reshape(dpoint.size, 1)
        dgrad = dgrad.reshape(dgrad.size, 1)
        p = 1 / np.sum(dgrad.T @ dpoint)
        prom1 = np.eye(dpoint.size) - p * dpoint @ dgrad.T
        prom2 = np.eye(dpoint.size) - p * dgrad @ dpoint.T
        prom3 = p * dpoint @ dpoint.T
        self.__C = prom1 @ self.__C @ prom2 + prom3
    
    @property
    def Oracle(self):
        return self.__oracle
    
    @property
    def Value(self):
        return self.__value
    
    @property
    def Point(self):
        return self.__point
    
def Projector(x):
    y = x - np.sum(x) * np.ones(x.size) / x.size
    return y

def StepChoser(prev, success):
    next = prev
    if success:
        if (next < 1.0):
            next *= 2.0
            if (next > 1):
                next = 1.0
    else:
        next /= 2.0
    return next 

def Checker(x):
    if (not np.all(x > 0)):
        return False
    else:
        return True
    
def PriorityMax(x, y):
    return x > y
    
class QuasiNewtonSpecializatorMax(QuasiNewtonIterator):
    def __init__(self, begin, p, pi):
        QuasiNewtonIterator.__init__(self, begin, Oracle(p, pi), Projector, StepChoser, PriorityMax, Checker, Normalization)
        
def FindMax(begin, p, pi, mistake):
    iterator = QuasiNewtonSpecializatorMax(begin, p, pi)
    prev = iterator.Value
    finished = False
    while(not finished):
        iterator.MakeStep()
        next = iterator.Value
        finished = (np.abs(next - prev) < mistake)
        prev = next
    return iterator

def GraphicMax(begin, p, pi, mistake):
    iterator = QuasiNewtonSpecializatorMax(begin, p, pi)
    iterations = np.array([iterator.Value])
    prev = iterator.Value
    finished = False
    while(not finished):
        iterator.MakeStep()
        iterations = np.append(iterations, iterator.Value)
        next = iterator.Value
        finished = (np.abs(next - prev) < mistake)
        prev = next
    plt.xlabel("Iteration")
    plt.ylabel("Difference with final value")
    plt.loglog(np.arange(iterations.size) + 1, np.abs(prev - iterations), "b-")
    plt.show()
    return iterator

def IterationsMistake(n, m, mistake):
    N = 10
    iterations_calc = np.zeros(0)
    iterations_grad = np.zeros(0)
    iterations_gess = np.zeros(0)
    iterations_time = np.zeros(0)
    for diff in mistake:
        calc = 0
        grad = 0
        gess = 0
        spend = 0.
        for i in range(N):
            p = np.abs(np.random.rand(n, m) * 5)
            pi = np.abs(np.random.rand(m))
            pi /= np.sum(pi)
            begin = np.ones(n) / n
            begin_time = time.time()
            iterator = FindMax(begin, p, pi, diff)
            end_time = time.time()
            calc += iterator.Oracle.Calculated
            grad += iterator.Oracle.CalculatedGradients
            gess += iterator.Oracle.CalculatedGessians
            spend += end_time - begin_time
        calc /= N
        grad /= N
        gess /= N
        spend /= N
        iterations_calc = np.append(iterations_calc, calc)
        iterations_grad = np.append(iterations_grad, grad)
        iterations_gess = np.append(iterations_gess, gess)
        iterations_time = np.append(iterations_time, spend)
    plt.xlabel("Mistake")
    plt.ylabel("Average number")
    plt.semilogx(mistake, iterations_grad)
    plt.legend(["Functions"])
    plt.show()
    plt.xlabel("Mistake")
    plt.ylabel("Average number")
    plt.semilogx(mistake, iterations_grad)
    plt.legend(["Gradients"])
    plt.show()
    plt.xlabel("Mistake")
    plt.ylabel("Average number")
    plt.semilogx(mistake, iterations_gess)
    plt.legend(["Gessians"])
    plt.show()
    plt.xlabel("Mistake")
    plt.ylabel("Average number")
    plt.semilogx(mistake, iterations_time)
    plt.legend(["Time"])
    plt.show()
    
def Test():
    n = 2
    m = 3
    p = np.array([[1, 2, 3], [10, 9, 8]])
    pi = np.array([0.5, 0.2, 0.3])
    begin = np.ones(n) / n
    iterator = GraphicMax(begin, p, pi, 10**(-10))
    print("Calculated:")
    print("Function: ", iterator.Oracle.Calculated)
    print("Gradient: ", iterator.Oracle.CalculatedGradients)
    print("Gessian: ", iterator.Oracle.CalculatedGessians)
    print("Point: ", iterator.Point)
    print("Value: ", iterator.Value)
    
def RandomTest():
    n = 10
    m = 10
    p = np.abs(np.random.rand(n, m) * 5)
    pi = np.abs(np.random.rand(m))
    pi /= np.sum(pi)
    begin = np.ones(n) / n
    iterator = GraphicMax(begin, p, pi, 10**(-10))
    print("Calculated:")
    print("Function: ", iterator.Oracle.Calculated)
    print("Gradient: ", iterator.Oracle.CalculatedGradients)
    print("Gessian: ", iterator.Oracle.CalculatedGessians)
    print("Point: ", iterator.Point)
    print("Value: ", iterator.Value)
    
Test()
RandomTest()
n = 10
m = 10
IterationsMistake(n, m, 10**(-1 * np.arange(1, 44) / 4.0))

<Figure size 640x480 with 1 Axes>

Calculated:
Function:  23
Gradient:  23
Gessian:  0
Point:  [3.94676206e-12 1.00000000e+00]
Value:  2.2145699244650876


<Figure size 640x480 with 1 Axes>

Calculated:
Function:  21
Gradient:  21
Gessian:  0
Point:  [1.39281799e-01 1.36283168e-01 1.79660782e-01 1.18365063e-01
 1.28419149e-02 7.46129311e-02 1.51962617e-01 6.41027870e-02
 1.22888938e-01 1.31549227e-11]
Value:  0.6812078389171701


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [2]:
dgrad = np.array([1, 2, 3])
dgrad = dgrad.reshape(dgrad.size, 1)
print(dgrad)
print(dgrad.T)

[[1]
 [2]
 [3]]
[[1 2 3]]
